# The big 6
 - SELECT
 - FROM
 - WHERE
 - GROUP BY
 - HAVING
 - ORDER BY


In [14]:
%%sh
# Open the sqlite3 CLI and create the database from the SQL file
rm -rf sample.db
sqlite3 sample.db < ./data/sample.sql

In [67]:
%%sh
sqlite3 sample.db <<EOF
.headers on
SELECT * FROM students
LIMIT 5
EOF

id|student_name|grade_level|gpa|school_lunch|birthday|email
1|Abby Johnson|10|3.1|Yes|2008-05-14|abby.johnson@mavenhighschool.com
2|Bob Smith|11|3.1|No|2007-09-30|bob.smith@mavenhighschool.com
3|Catherine Davis|12|3.6|Yes|2006-11-21|catherine.davis@mavenhighschool.com
4|Daniel Brown|9|3.5|Yes|2009-03-10|daniel.brown@mavenhighschool.com
5|Eva Martinez|10|2.7|No|2008-02-05|eva.martinez@mavenhighschool.com


## Show students with gpa over 3.5

In [59]:
%%sh
sqlite3 sample.db <<EOF
.headers on
SELECT student_name, gpa, school_lunch
FROM students 
WHERE gpa > 3.5
EOF

student_name|gpa|school_lunch
Catherine Davis|3.6|Yes
Liam Green|4|Yes
Olivia Adams|3.7|Yes


## Sort the students by GPA (ORDER BY)

In [62]:
%%sh
sqlite3 sample.db <<EOF
.headers on
SELECT student_name, gpa, school_lunch
FROM students
WHERE school_lunch = 'Yes' AND gpa > 3.5
ORDER BY gpa DESC
EOF

student_name|gpa|school_lunch
Liam Green|4|Yes
Olivia Adams|3.7|Yes
Catherine Davis|3.6|Yes


## Show the avarage gpa for each grade lavel (GROUP BY)

In [65]:
%%sh
sqlite3 sample.db <<EOF
.headers on
SELECT grade_level, AVG(gpa) AS avg_gpa
FROM students
GROUP BY grade_level
ORDER BY grade_level ASC
EOF

grade_level|avg_gpa
9|3.4
10|3.15
11|3.05
12|3.16666666666667


## Show the grade levels with an avarage gpa bellow 3.3 (HAVING)

In [58]:
%%sh
sqlite3 sample.db <<EOF
.headers on
SELECT grade_level, AVG(gpa) AS avg_gpa
FROM students
GROUP BY grade_level
HAVING AVG(gpa) < 3.3
ORDER BY grade_level;
EOF

grade_level|avg_gpa
10|3.15
11|3.05
12|3.16666666666667


# JOIN

In [82]:
%%sh

sqlite3 sample.db <<EOF
.headers on
SELECT * FROM students LIMIT 1
EOF

sqlite3 sample.db <<EOF
.headers on
SELECT * FROM student_grades LIMIT 1
EOF


id|student_name|grade_level|gpa|school_lunch|birthday|email
1|Abby Johnson|10|3.1|Yes|2008-05-14|abby.johnson@mavenhighschool.com
semester_id|class_id|department|class_name|student_id|final_grade
SPR_2024|101|Math|Algebra|4|85


In [81]:
%%sh

sqlite3 sample.db <<EOF
.headers on
SELECT students.id, students.student_name, 
    students.gpa, student_grades.final_grade
FROM students LEFT JOIN student_grades
    ON students.id = student_grades.student_id
-- LIMIT 5
EOF

id|student_name|gpa|final_grade
1|Abby Johnson|3.1|82
1|Abby Johnson|3.1|85
1|Abby Johnson|3.1|93
1|Abby Johnson|3.1|94
2|Bob Smith|3.1|75
2|Bob Smith|3.1|80
2|Bob Smith|3.1|87
2|Bob Smith|3.1|88
3|Catherine Davis|3.6|92
3|Catherine Davis|3.6|95
3|Catherine Davis|3.6|98
3|Catherine Davis|3.6|100
4|Daniel Brown|3.5|75
4|Daniel Brown|3.5|82
4|Daniel Brown|3.5|85
4|Daniel Brown|3.5|95
5|Eva Martinez|2.7|74
5|Eva Martinez|2.7|80
5|Eva Martinez|2.7|85
5|Eva Martinez|2.7|87
6|Frank Wilson|3.2|90
6|Frank Wilson|3.2|95
6|Frank Wilson|3.2|95
6|Frank Wilson|3.2|98
7|Grace Lee|3|82
7|Grace Lee|3|86
7|Grace Lee|3|94
7|Grace Lee|3|98
8|Henry Taylor|3|72
8|Henry Taylor|3|76
8|Henry Taylor|3|80
8|Henry Taylor|3|90
9|Isabella Moore|2.8|72
9|Isabella Moore|2.8|76
9|Isabella Moore|2.8|84
9|Isabella Moore|2.8|90
10|Jack Thompson|2.9|77
10|Jack Thompson|2.9|82
10|Jack Thompson|2.9|84
10|Jack Thompson|2.9|85
11|Karen White|3.4|87
11|Karen White|3.4|90
11|Karen White|3.4|95
11|Karen White|3.4|98
12|Liam Gre

In [ ]:
## Window Functions
> Window functions are a way to perform calculations across a set of table rows that are related to the current row.

In [3]:
%%sh

sqlite3 sample.db <<EOF
.headers on
SELECT student_name, gpa, 
    ROW_NUMBER() OVER(ORDER BY gpa DESC) AS row_num
-- /* window function */
FROM students
ORDER BY gpa DESC
LIMIT 5
EOF

student_name|gpa|row_num
Liam Green|4|1
Olivia Adams|3.7|2
Catherine Davis|3.6|3
Daniel Brown|3.5|4
Karen White|3.4|5


In [9]:
%%sh

sqlite3 sample.db <<EOF
.headers on
SELECT student_name, gpa, 
    ROW_NUMBER() OVER(ORDER BY gpa DESC) AS row_num,
    RANK() OVER(ORDER BY gpa DESC) AS rank_num,
    DENSE_RANK() OVER(ORDER BY gpa DESC) AS dense_rank_num
FROM students
ORDER BY gpa DESC
LIMIT 5
EOF

student_name|gpa|row_num|rank_num|dense_rank_num
Liam Green|4|1|1|1
Olivia Adams|3.7|2|2|2
Catherine Davis|3.6|3|3|3
Daniel Brown|3.5|4|4|4
Karen White|3.4|5|5|5


# Common Table Expression (CTE) 

In [22]:
%%sh

sqlite3 sample.db <<EOF
.headers on

SELECT student_id, class_id, AVG(final_grade ) AS avg_final_grade
FROM student_grades
GROUP BY class_id

EOF

student_id|class_id|avg_final_grade
4|101|87.0
1|102|81.6666666666667
2|103|88.8
3|104|85.0
4|201|84.25
1|202|85.8
3|203|85.0
1|301|86.4285714285714
2|302|88.25
1|401|90.0
3|404|97.6666666666667


In [21]:
%%sh

sqlite3 sample.db <<EOF
.headers on

WITH ClassAverageGPA AS (
    SELECT student_id, class_id, AVG(final_grade ) AS avg_final_grade
    FROM student_grades
    GROUP BY class_id
)

SELECT s.id, s.student_name, s.grade_level, c.avg_final_grade
FROM students s
JOIN ClassAverageGPA c ON s.id = c.student_id
-- WHERE s.grade_level > c.avg_final_grade
ORDER BY s.grade_level DESC
EOF

id|student_name|grade_level|avg_final_grade
3|Catherine Davis|12|85.0
3|Catherine Davis|12|85.0
3|Catherine Davis|12|97.6666666666667
2|Bob Smith|11|88.8
2|Bob Smith|11|88.25
1|Abby Johnson|10|81.6666666666667
1|Abby Johnson|10|85.8
1|Abby Johnson|10|86.4285714285714
1|Abby Johnson|10|90.0
4|Daniel Brown|9|87.0
4|Daniel Brown|9|84.25
